세세분류

In [ ]:
import pandas as pd
import numpy as np

from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
train=pd.read_csv('/content/drive/MyDrive/애쓰는 감자/data/train.csv',dtype={'KEDCD':str,'BZ_PPOS_ITM_CTT':str,'KSIC10_BZC_CD':str,'mid':str,'Big':str})
plus = pd.read_csv('/content/drive/MyDrive/애쓰는 감자/data/Augmentation_data.csv')

In [ ]:
import re
plus["BZ_PPOS_ITM_CTT"] = plus["Dis"].str.replace('[^\w\s]', '', regex=True)
plus.head()

,Dis,Target,BZ_PPOS_ITM_CTT
0,"농업, 임업 및 어업(01~03)",A,농업 임업 및 어업0103
1,광업(05~08),B,광업0508
2,제조업(10~34),C,제조업1034
3,"전기, 가스, 증기 및 공기 조절 공급업(35)",D,전기 가스 증기 및 공기 조절 공급업35
4,"수도, 하수 및 폐기물 처리, 원료 재생업(36~39)",E,수도 하수 및 폐기물 처리 원료 재생업3639


In [ ]:
plus["BZ_PPOS_ITM_CTT"] = plus["BZ_PPOS_ITM_CTT"].str.replace('[0-9]+', '', regex=True)
plus.head()

,Dis,Target,BZ_PPOS_ITM_CTT
0,"농업, 임업 및 어업(01~03)",A,농업 임업 및 어업
1,광업(05~08),B,광업
2,제조업(10~34),C,제조업
3,"전기, 가스, 증기 및 공기 조절 공급업(35)",D,전기 가스 증기 및 공기 조절 공급업
4,"수도, 하수 및 폐기물 처리, 원료 재생업(36~39)",E,수도 하수 및 폐기물 처리 원료 재생업


In [ ]:
plus["sese"] = plus["Target"].str[1:]
plus

,Dis,Target,BZ_PPOS_ITM_CTT,sese
0,"농업, 임업 및 어업(01~03)",A,농업 임업 및 어업,
1,광업(05~08),B,광업,
2,제조업(10~34),C,제조업,
3,"전기, 가스, 증기 및 공기 조절 공급업(35)",D,전기 가스 증기 및 공기 조절 공급업,
4,"수도, 하수 및 폐기물 처리, 원료 재생업(36~39)",E,수도 하수 및 폐기물 처리 원료 재생업,
...,...,...,...,...
2016,가구 내 고용활동,T97000,가구 내 고용활동,97000
2017,자가 소비를 위한 가사 생산 활동,T98100,자가 소비를 위한 가사 생산 활동,98100
2018,자가 소비를 위한 가사 서비스 활동,T98200,자가 소비를 위한 가사 서비스 활동,98200
2019,주한 외국 공관,U99001,주한 외국 공관,99001


In [ ]:
# 추가내용 파일 - 해당 분류에 맞게 삭제 
for i in range(len(plus)):
  if len(plus['sese'][i])<5:
    plus=plus.drop(index=i)

print(len(plus))

1196


In [ ]:
plus['sese'].unique()

array(['01110', '01121', '01122', ..., '98200', '99001', '99009'],
      dtype=object)

In [ ]:
data_sese = plus.loc[:,['BZ_PPOS_ITM_CTT', 'sese']]
data_sese.tail()

,BZ_PPOS_ITM_CTT,sese
2016,가구 내 고용활동,97000
2017,자가 소비를 위한 가사 생산 활동,98100
2018,자가 소비를 위한 가사 서비스 활동,98200
2019,주한 외국 공관,99001
2020,기타 국제 및 외국기관,99009


In [ ]:
for i in range(len(train)):
  if len(train['KSIC10_BZC_CD'][i])<6:
    train=train.drop(index=i)

print(len(train))

In [ ]:
train['sese']=train['sese'].str[1:6]
train.head()

In [ ]:
trn = pd.concat([train, data_sese], ignore_index = True)
trn.tail()

In [ ]:
trn = trn.loc[:,['BZ_PPOS_ITM_CTT', 'sese']]
trn.tail()

In [ ]:
trn =trn.dropna(axis=0)
len(trn)

In [ ]:
trn_shuffle = trn.sample(frac=1).reset_index(drop=True)
trn_shuffle.tail()

In [ ]:
trn_shuffle.to_csv('/content/drive/MyDrive/애쓰는 감자/data/mid_data/train_data_sese.csv', index = False)

In [ ]:
# 라벨링
from sklearn.preprocessing import LabelEncoder
encoder = LabelEncoder()
encoder.fit(trn_shuffle['sese'])
trn_shuffle['sese_idx'] = encoder.transform(trn_shuffle['sese'])
trn_shuffle.head()

In [ ]:
# 라벨링된 것 매핑
mapping = dict(zip(range(len(encoder.classes_)), encoder.classes_))
mapping

In [ ]:
trn_shuffle = trn_shuffle.loc[:,['BZ_PPOS_ITM_CTT', 'sese_idx']]
trn_shuffle.head()

In [ ]:
trn_shuffle.to_csv('/content/drive/MyDrive/애쓰는 감자/data/mid_data/train_sese.txt', sep='\t', index=True)